In [1]:
import torch 
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
    "vocab_size":50257,
    "context_length":1024,
    "embedding_dim":768,
    "num_of_heads":12,
    "num_of_layers":12,
    "drop_rate_attention":0.1,
    "drop_rate_embedding":0.1,
    "drop_rate_shortcut":0.1,
    "qkv_bias":False
}

In [3]:

class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_of_head,qkv_bias=False):
        super().__init__()
        
        assert (d_out % num_of_head == 0), "d_out must be divisible by num_of_head"
        
        self.d_out = d_out                                   
        self.dropout = nn.Dropout(dropout)
        self.num_of_head = num_of_head                         
        
        self.head_dim = d_out // num_of_head                
        self.out_proj = nn.Linear(d_out, d_out)             
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length,context_length), diagonal=1)
        )
        
    def forward(self, x):
        num_of_inputs, num_of_tokens, embedding_dim = x.shape           
        
        queries = self.W_query(x)                            
        keys = self.W_key(x)
        values = self.W_value(x)
        
        queries = queries.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)      
        keys = keys.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       
        values = values.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       
        
        
        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        
        attention_score = queries @ keys.transpose(2,3)
        mask_bool = self.mask.bool() [:num_of_tokens, :num_of_tokens]
        
        attention_score.masked_fill_(mask_bool, -torch.inf)
        
        k_d = keys.shape[-1]
        attention_weight = torch.softmax(attention_score / k_d**0.5, dim=-1)
        attention_weight = self.dropout(attention_weight)
        
        context_vector = (attention_weight @ values).transpose(1,2)
        
        context_vector = context_vector.contiguous().view(
            num_of_inputs, num_of_tokens, self.d_out
        )
        context_vector = self.out_proj(context_vector)
        return context_vector

In [4]:
class LayerNormalization(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(embedding_dim))
        self.shift = nn.Parameter(torch.zeros(embedding_dim))
        
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        variance = x.var(dim=-1, keepdim=True)
        
        x_normalized = (x - mean)/ torch.sqrt(variance + self.eps)
        return self.scale * x_normalized + self.shift

In [5]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,x):
        return 0.5 * x * (
            1 + torch.tanh(torch.sqrt(torch.tensor(2.0/torch.pi)) * (x + 0.44715 * torch.pow(x,3)))
        )

In [6]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(config['embedding_dim'], 4* config['embedding_dim']),
            GELU(),
            nn.Linear(4*config['embedding_dim'],config['embedding_dim'])
        )
        
    def forward(self, x):
        return self.layers(x)

In [7]:
inputs = torch.rand(2,4,768)

In [8]:
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm1 = LayerNormalization(config['embedding_dim'])
        self.attention = MultiHeadAttention(
            d_in=config['embedding_dim'],
            d_out=config['embedding_dim'],
            context_length=config['context_length'],
            dropout=config['drop_rate_attention'],
            num_of_head=config['num_of_heads'],
            qkv_bias=config['qkv_bias']
        )
        self.dropout_shortcut = nn.Dropout(config['drop_rate_shortcut'])
        self.layer_norm2 = LayerNormalization(config['embedding_dim'])     
        self.ff = FeedForward(config)
        
    def forward(self,x):
        shortcut = x
        
        x = self.layer_norm1(x)
        x = self.attention(x)
        x = self.dropout_shortcut(x)
        
        x = x + shortcut
        
        shortcut = x 
        x = self.layer_norm2(x)
        x = self.ff(x)
        x = self.dropout_shortcut(x)
        
        x = x + shortcut
        return x

In [9]:
transformerBlock = TransformerBlock(GPT_CONFIG_124M) 

In [10]:
transformerBlock

TransformerBlock(
  (layer_norm1): LayerNormalization()
  (attention): MultiHeadAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=768, out_features=768, bias=True)
    (W_query): Linear(in_features=768, out_features=768, bias=False)
    (W_key): Linear(in_features=768, out_features=768, bias=False)
    (W_value): Linear(in_features=768, out_features=768, bias=False)
  )
  (dropout_shortcut): Dropout(p=0.1, inplace=False)
  (layer_norm2): LayerNormalization()
  (ff): FeedForward(
    (layers): Sequential(
      (0): Linear(in_features=768, out_features=3072, bias=True)
      (1): GELU()
      (2): Linear(in_features=3072, out_features=768, bias=True)
    )
  )
)

In [11]:
output = transformerBlock(inputs)
output

tensor([[[ 7.6583e-01, -5.6543e-01,  1.1176e-01,  ...,  6.2051e-01,
           8.4087e-01, -5.0407e-01],
         [ 4.2186e-01,  3.0178e-02, -1.3766e-01,  ...,  1.0595e+00,
          -1.9451e-01, -7.9429e-02],
         [ 3.4338e-01, -1.5067e-01,  1.7121e-01,  ...,  2.1207e-01,
           5.4339e-01,  4.7946e-01],
         [ 8.1102e-01,  1.8683e-01,  1.6864e-01,  ...,  5.5555e-01,
           2.6048e-01,  1.4430e-01]],

        [[-8.4855e-02,  1.0628e+00,  6.9442e-01,  ...,  1.1393e+00,
           1.1212e+00,  1.6567e+00],
         [-2.5907e-02,  8.0964e-01,  7.8358e-01,  ...,  1.0527e+00,
           4.2865e-01,  5.0622e-01],
         [ 8.3885e-01, -1.4489e-03,  1.2695e+00,  ...,  4.9450e-01,
           7.9143e-01,  2.2807e-01],
         [ 6.7424e-01,  7.7560e-01,  5.1858e-01,  ...,  1.2085e-01,
           5.1988e-01,  6.3220e-01]]], grad_fn=<AddBackward0>)

In [12]:
print(inputs.shape, output.shape)

torch.Size([2, 4, 768]) torch.Size([2, 4, 768])


In [13]:
class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.token_embedding = nn.Embedding(config['vocab_size'], config['embedding_dim'])
        self.positional_embedding = nn.Embedding(config['context_length'], config['embedding_dim'])
        self.dropout_embedding = nn.Dropout(config['drop_rate_embedding'])
        
        self.transformer_blocks = nn.Sequential(
            *[TransformerBlock(config) for _ in range(config['num_of_layers'])]
        )
        
        self.final_normalization = LayerNormalization(config['embedding_dim'])
        
        self.output_head = nn.Linear(config['embedding_dim'], config['vocab_size'], bias=False)
        
        
    def forward(self,inputs):
        batch_size, no_of_token = inputs.shape
        token_embedding = self.token_embedding(inputs)
        positional_embedding = self.positional_embedding(
            torch.arange(no_of_token, device=inputs.device)
        )
        
        x = token_embedding + positional_embedding
        x = self.dropout_embedding(x)
        x = self.transformer_blocks(x)
        x = self.final_normalization(x)
        logits = self.output_head(x)
        return logits
    
        

In [14]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')

batch = []

tx1 = "Every efforts moves you"
tx2 = "Everyday hold a"

batch.append(torch.tensor(tokenizer.encode(tx1)))
batch.append(torch.tensor(tokenizer.encode(tx2)))
batch = torch.stack(batch, dim=0)
batch

tensor([[6109, 4040, 6100,  345],
        [6109,  820, 1745,  257]])

In [15]:
batch.shape

torch.Size([2, 4])

In [16]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
logits = model(batch)
logits

tensor([[[ 0.0354, -0.4721, -0.4587,  ..., -0.2188, -0.7655, -0.5078],
         [ 1.2091, -0.7111, -0.3461,  ..., -0.4756,  0.1423,  0.4917],
         [ 1.1188, -0.1816, -0.4621,  ...,  0.1599, -0.6646,  0.0516],
         [-0.4757,  0.1577, -0.7076,  ...,  0.9585,  0.4032, -0.4800]],

        [[-0.3335, -0.5786, -0.7396,  ..., -0.3007, -0.6175, -0.5974],
         [ 0.6615, -0.0245,  0.3522,  ..., -0.0117, -0.3577,  0.2663],
         [ 1.4650, -0.7124, -1.0607,  ...,  0.0926, -0.4379,  0.1451],
         [-0.0892, -0.3618,  0.2950,  ...,  1.1519, -0.6237,  0.1250]]],
       grad_fn=<UnsafeViewBackward0>)

In [17]:
logits.shape

torch.Size([2, 4, 50257])